In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pydantic import BaseModel, Field
from typing import Dict, List
from langchain_community.tools import DuckDuckGoSearchResults

In [3]:
# 상태 정보 정의

class State(BaseModel):
    company_name: str = Field(description="회사 이름")
    segment: Dict[str, dict] = Field(description="사업부 정보", default_factory=dict)
    PER: float = Field(description="기업의 현재 PER(TTM)", default=0)
    peer_list: List[str] = Field(description="경쟁사 리스트", default_factory=list)
    average_peer_PER: float = Field(description="경쟁사 PER 평균", default=0)
    result: Dict[str, str] = Field(description="결과", default_factory=dict)


In [4]:
# 상태 정보 초기화
company_name = "네이버"
current_quarter_sales_consensus = 4935.0 * (10 ** 8) # 크롤링 추가 필요 (10^8 =억원)

In [5]:
state = State(company_name=company_name)

# 데이터 로드 TODO: 사업보고서 혹은 애널리스트 리포트 retrieve해서 데이터 로드
import pandas as pd

income_stmt_cum = pd.read_csv(f"./docs/{company_name}_segment_sales+income_stmt_cum.csv",header=0)

# 사업부 추출
segments = []
for item in income_stmt_cum.iloc[:, 0]:
    if item == '영업수익':
        break
    segments.append(item)

# YoY 계산 ((현재 - 이전) / 이전) * 100
present_quarter = income_stmt_cum.iloc[:,1]
year_ago_quarter = income_stmt_cum.iloc[:,1+4]
yoy = ((present_quarter - year_ago_quarter) / year_ago_quarter) * 100

# 각 사업부별 YoY와 사업부 이름을 함께 보여주기
for segment, growth in zip(income_stmt_cum['계정'], yoy):
    if segment in segments:  # segments 리스트에 있는 사업부만 출력
        state.segment[segment] = {"yoy":growth}

# 영업수익 yoy 계산
for segment, growth in zip(income_stmt_cum['계정'], yoy):
    if segment == '영업수익':  # segments 리스트에 있는 사업부만 출력
        state.segment['영업수익'] = {"yoy":growth}

# 사업부별 매출액 추출
for segment, sales in zip(income_stmt_cum['계정'], present_quarter):
    if segment in segments:
        state.segment[segment].update({"sales":sales})

# 영업수익 추출
for segment, sales in zip(income_stmt_cum['계정'], present_quarter):
    if segment == '영업수익':  
        state.segment['영업수익'].update({"sales":sales})

print(state.segment)


{'서치플랫폼': {'yoy': 8.296626477043807, 'sales': 2881470000000.0}, '커머스': {'yoy': 13.873396246421377, 'sales': 2147880000000.0}, '핀테크': {'yoy': 10.88772053839353, 'sales': 1107490000000.0}, '콘텐츠': {'yoy': 4.928787974483673, 'sales': 1329070000000.0}, '클라우드': {'yoy': 20.18516789593253, 'sales': 386191000000.0}, '영업수익': {'yoy': 10.071436181248284, 'sales': 7852100000000.0}}


In [6]:
# 사업부별 뉴스 검색
duckduckgo_search_tool = DuckDuckGoSearchResults(max_results=1,backend="news")
for segment in segments:
    news_result = duckduckgo_search_tool.invoke(
        {
            "query": f"{state.company_name} {segment} 사업부 매출"
        }
    )
    state.segment[segment].update({"news_result":news_result})

print(state.segment)

{'서치플랫폼': {'yoy': 8.296626477043807, 'sales': 2881470000000.0, 'news_result': 'snippet: 네이버가 최수연 대표 취임 3년차인 올해 매출 10조원을 ... 최대치를 기록했다. 네이버 광고 사업 성과가 포함된 서치 플랫폼 매출이 성과를 견인했다., title: 네이버, 최수연號 3년차 매출 10조 돌파 유력…\'플랫폼 역량 강화\' 통했다, link: https://www.etnews.com/20241108000168, date: 2024-11-10T15:00:00+00:00, source: Etnews.com, snippet: 사업 부문별 매출액은 서치플랫폼 ... 네이버 측은 브랜드와의 협업과 멤버십 혜택 강화, 배송 품질 개선 등에 따른 거래액 상승과 브랜드 솔루션 패키지, 도착보장 사용률 증가가 커머스 매출 ..., title: 네이버, 3분기 영업익 사상 최대 실적…"검색 등 전 사업 고른 성장", link: https://news.mtn.co.kr/news-detail/2024110808322468489, date: 2024-11-08T08:54:00+00:00, source: MTN 뉴스, snippet: 네이버 3분기 실적. / 네이버 네이버는 3분기 매출 2조7156억원 ... 사업 부문별 매출액은 서치플랫폼 9977억원 커머스 7254억원 핀테크 3851 ..., title: 네이버 \'사상 최대\' 영업익 기록… 3분기 5253억원, link: https://it.chosun.com/news/articleView.html?idxno=2023092126781, date: 2024-11-08T09:52:00+00:00, source: 조선일보, snippet: 네이버가 홈피드, 클립 등 모바일 애플리케이션(앱) 개편 효과에 따른 광고 매출 증가로 ... 부문별 매출액은 서치 플랫폼 9977억원 커머스 ..., title: 앱 개편으로 역대급 실적 쓴 네이버..."가속 성장 이어간다

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class yoy_prediction(BaseModel):
    """yoy 예측값을 나타내는 모델"""
    business_segment: str = Field(description="사업부")
    yoy: float = Field(description="사업부 yoy 예측값")
    reason: str = Field(description="사업부 yoy 예측값의 근거")

parser = PydanticOutputParser(pydantic_object=yoy_prediction)


template = """
당신은 증권사 소속 애널리스트입니다.
다음 내용을 참고하여 {company_name}의 {business_segment} 사업부의 매출 혹은 비용이 yoy로 얼마나 변할지 예측하시오.

직전 분기 yoy : {yoy}

{company_name}의 {business_segment} 사업부의 매출 혹은 비용에 큰 영향을 미치는 뉴스 : {news}

응답 형식:
{format_instructions}
"""

for segment in segments:
    prompt = ChatPromptTemplate.from_template(template=template, partial_variables={"format_instructions": parser.get_format_instructions()})

    llm = ChatOpenAI(temperature=0, model="gpt-4o")

    chain = prompt | llm | parser

    result = chain.invoke({"company_name": state.company_name, "news": state.segment[segment]['news_result'], "business_segment": segment, "yoy": state.segment[segment]['yoy']})

    # 예측값 및 예측 이유업데이트
    state.segment[segment].update({"yoy_prediction":result.yoy})
    state.segment[segment].update({"yoy_prediction_reason":result.reason})

print(state.segment)

{'서치플랫폼': {'yoy': 8.296626477043807, 'sales': 2881470000000.0, 'news_result': 'snippet: 네이버가 최수연 대표 취임 3년차인 올해 매출 10조원을 ... 최대치를 기록했다. 네이버 광고 사업 성과가 포함된 서치 플랫폼 매출이 성과를 견인했다., title: 네이버, 최수연號 3년차 매출 10조 돌파 유력…\'플랫폼 역량 강화\' 통했다, link: https://www.etnews.com/20241108000168, date: 2024-11-10T15:00:00+00:00, source: Etnews.com, snippet: 사업 부문별 매출액은 서치플랫폼 ... 네이버 측은 브랜드와의 협업과 멤버십 혜택 강화, 배송 품질 개선 등에 따른 거래액 상승과 브랜드 솔루션 패키지, 도착보장 사용률 증가가 커머스 매출 ..., title: 네이버, 3분기 영업익 사상 최대 실적…"검색 등 전 사업 고른 성장", link: https://news.mtn.co.kr/news-detail/2024110808322468489, date: 2024-11-08T08:54:00+00:00, source: MTN 뉴스, snippet: 네이버 3분기 실적. / 네이버 네이버는 3분기 매출 2조7156억원 ... 사업 부문별 매출액은 서치플랫폼 9977억원 커머스 7254억원 핀테크 3851 ..., title: 네이버 \'사상 최대\' 영업익 기록… 3분기 5253억원, link: https://it.chosun.com/news/articleView.html?idxno=2023092126781, date: 2024-11-08T09:52:00+00:00, source: 조선일보, snippet: 네이버가 홈피드, 클립 등 모바일 애플리케이션(앱) 개편 효과에 따른 광고 매출 증가로 ... 부문별 매출액은 서치 플랫폼 9977억원 커머스 ..., title: 앱 개편으로 역대급 실적 쓴 네이버..."가속 성장 이어간다

In [8]:
# 다음 분기(예측) 열 추가 (모든 행 0으로 초기화)
income_stmt_cum['next_quarter'] = 0

# segments에 있는 계정에 대해서만 yoy_prediction을 적용하여 업데이트
for segment in segments:
    yoy_prediction = state.segment[segment]['yoy_prediction']
    predicted_growth_rate = 1 + (yoy_prediction / 100)  # yoy_prediction을 성장률로 변환
    
    # 해당 segment 행의 next_quarter 값을 업데이트(next_quarter의 1년 전 분기 값 * 성장률)
    income_stmt_cum.loc[income_stmt_cum['계정'] == segment, 'next_quarter'] = income_stmt_cum.loc[income_stmt_cum['계정'] == segment].iloc[:,4] * predicted_growth_rate

# next_quarter의 segments 행들의 합계 계산
revenue_sum = income_stmt_cum.loc[income_stmt_cum['계정'].isin(segments), 'next_quarter'].sum()

# segments의 합계를 영업수익 값으로 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익', 'next_quarter'] = revenue_sum

# 직전분기의 영업수익 대비 영업비용 비율 계산
revenue_current_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익'].iloc[:,1].values[0]
cost_current_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업비용'].iloc[:,1].values[0]
cost_ratio = cost_current_quarter / revenue_current_quarter

# 동기간 전분기의 영업비용비율을 다음 분기의 영업비용 예측에 사용(영업이익률을 그대로 사용)
revenue_next_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익', 'next_quarter'].values[0]
cost_next_quarter = revenue_next_quarter * cost_ratio

# 영업비용 행의 next_quarter 값을 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '영업비용', 'next_quarter'] = cost_next_quarter

# next_quarter의 영업이익 계산 (영업수익 - 영업비용)
operating_profit_next_quarter = revenue_next_quarter - cost_next_quarter

# 영업이익 행의 next_quarter 값을 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '영업이익', 'next_quarter'] = operating_profit_next_quarter

# 다음분기의 영업이익률 계산 (영업이익/영업수익 * 100)
operating_margin_next_quarter = (operating_profit_next_quarter / revenue_next_quarter)

# 영업이익률 행의 next_quarter 값을 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '영업이익률', 'next_quarter'] = operating_margin_next_quarter

# 직전분기의 순이익률 계산
net_income_current_quarter = income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익'].iloc[:,1].values[0]
net_margin_current_quarter = net_income_current_quarter / revenue_current_quarter

# 다음분기의 순이익 계산(직전분기의 순이익률 * 다음분기의 추정 영업수익)
net_income_next_quarter = revenue_next_quarter * net_margin_current_quarter

# 순이익과 순이익률 행 업데이트
income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익', 'next_quarter'] = net_income_next_quarter
income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익률', 'next_quarter'] = net_margin_current_quarter

income_stmt_cum

/var/folders/f7/84_7pkk50r10snsc2qr066600000gn/T/ipykernel_51861/4178160815.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.7763045e+12]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  income_stmt_cum.loc[income_stmt_cum['계정'] == segment, 'next_quarter'] = income_stmt_cum.loc[income_stmt_cum['계정'] == segment].iloc[:,4] * predicted_growth_rate


,계정,2024-3Q,2024-2Q,2024-1Q,2023-4Q,2023-3Q,next_quarter
0,서치플랫폼,2.881470e+12,1.883760e+12,9.053540e+11,3.589060e+12,2.660720e+12,3.965911e+12
1,커머스,2.147880e+12,1.422500e+12,7.034510e+11,2.546650e+12,1.886200e+12,2.941381e+12
2,핀테크,1.107490e+12,7.223740e+11,3.539100e+11,1.354770e+12,9.987490e+11,1.524116e+12
3,콘텐츠,1.329070e+12,8.663160e+11,4.463190e+11,1.732980e+12,1.266640e+12,1.776304e+12
4,클라우드,3.861910e+11,2.415980e+11,1.170200e+11,4.471840e+11,3.213300e+11,5.478004e+11
5,영업수익,7.852100e+12,5.136540e+12,2.526060e+12,9.670640e+12,7.133640e+12,1.075551e+13
6,영업비용,6.414850e+12,4.224560e+12,2.086760e+12,8.181820e+12,6.050310e+12,8.786822e+12
7,영업이익,1.437250e+12,9.119780e+11,4.392930e+11,1.488820e+12,1.083330e+12,1.968691e+12
8,영업이익률,1.830400e-01,1.775471e-01,1.739048e-01,1.539526e-01,1.518621e-01,1.830402e-01
9,순이익,1.417940e+12,8.878600e+11,5.558030e+11,9.850180e+11,6.865770e+11,1.942241e+12


In [9]:
# income_stmt_cum 데이터프레임을 엑셀 파일로 저장
income_stmt_cum.to_excel('income_stmt_cum.xlsx', index=False)

In [ ]:
#직전분기 매출액(누적) 컨센서스와 실제값 저장

def naver_sales_earning_surprise(current_quarter_sales_consensus:float) -> float:
    """네이버 증권 -> 종목분석 -> 컨센서스 -> 어닝서프라이즈(매출액) -> 분기
        return :
        직전 분기 매출액에 대한 발표직전 컨센서스 (단위: 억원) + 직전 분기의 전분기까지의 실제 매출액 = 직전 분기까지 누적 매출액 컨센서스
    """
    
    before_current_quarter_sales_cum = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익'].iloc[:,2].values[0]
    current_quarter_sales_cum_consensus = current_quarter_sales_consensus + before_current_quarter_sales_cum

    return current_quarter_sales_cum_consensus

current_quarter_sales_cum_consensus = naver_sales_earning_surprise(current_quarter_sales_consensus)
current_quarter_sales_cum = state.segment['영업수익']['sales']

year_ago_quarter_sales_cum = income_stmt_cum.loc[income_stmt_cum['계정'] == '영업수익'].iloc[:,5].values[0]
yoy_consensus = ((current_quarter_sales_cum_consensus - year_ago_quarter_sales_cum) / year_ago_quarter_sales_cum) * 10

state.segment['영업수익'].update({"yoy_consensus":yoy_consensus})
state.segment['영업수익'].update({"sales_consensus":current_quarter_sales_cum_consensus})

state.segment['영업수익']

In [ ]:
def current_quarter_review(state:State) -> None:
    """직전 분기 실적 리뷰를 작성"""

    result_of_business_segment = ""
    for segment in segments:
        result_of_business_segment += f"{segment} 사업부의 직전 분기 매출액은 {state.segment[segment]['sales']} 입니다. yoy로 {state.segment[segment]['yoy']}% 변했습니다.\n"

    result_of_total_business = f"총 매출액은 {state.segment['영업수익']['sales']} 입니다. yoy로 {state.segment['영업수익']['yoy']}% 변했습니다."

    consensus_of_total_business = f"총 매출액에 대한 컨센서스는 {state.segment['영업수익']['sales_consensus']} 였습니다. yoy로 {state.segment['영업수익']['yoy_consensus']}% 변화하는 것이 컨센서스였습니다."

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 직전 분기 실적에 대한 리뷰를 작성하시오.

    직전 분기 사업부별 매출 실적 : {result_of_business_segment}
    직전 분기 총 매출 실적 : {result_of_total_business}
    직전 분기 총 매출 실적에 대한 컨센서스 : {consensus_of_total_business}

    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm 
    review = chain.invoke({"result_of_business_segment":result_of_business_segment, "result_of_total_business":result_of_total_business, "consensus_of_total_business":consensus_of_total_business})
    state.result.update({"sales_review":review.content})

    return None

current_quarter_review(state)
print(state.result['sales_review'])

In [11]:
# peer PER 찾기 위한 티커 찾기
from yahooquery import search
from deep_translator import GoogleTranslator

# 한글 문자 범위를 이용해 한글 포함 여부 확인
def contains_korean(text):
    for char in text:
        if '\uac00' <= char <= '\ud7a3' or '\u3131' <= char <= '\u318e':
            return True
    return False

def get_ticker(company_name):
    try:            
        # 한글 포함 여부 확인
        is_korean = contains_korean(company_name)
        if is_korean:
            # 회사명을 영어로 번역
            translated = GoogleTranslator(source='auto', target='en').translate(company_name)
            
            # 번역된 이름으로 검색
            results = search(translated)
        else:
            results = search(company_name)
            
        # KSC, NYSE, NASDAQ, AMEX, JPX, HKG 순서로 찾기
        for quote in results['quotes']:
            if quote['exchange'] == 'KSC': # 한국
                return quote['symbol']
            elif quote['exchange'] == 'NYQ': # NYSE
                return quote['symbol']
            elif quote['exchange'] == 'NMS': # NASDAQ
                return quote['symbol']
            elif quote['exchange'] == 'JPX': # 일본
                return quote['symbol']
            elif quote['exchange'] == 'HKG': # 홍콩
                return quote['symbol']
            else:
                continue
        
        # KSC, NYSE, NASDAQ, AMEX에 없으면 None 반환
        return None
    except Exception as e:
        print(f"Error translating or searching for {company_name}: {e}")
        return None

In [12]:
# StructuredOutputParser(peer 리스트 출력)

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변, 파이썬 리스트 형식이어야 함."),
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [13]:
import yfinance as yf
import numpy as np

def find_peer(company: str) -> list[str]:
    prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
    )
    chain = prompt | llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결
    peer_list = chain.invoke({"question": f"{company}와 사업구조가 비슷하고, 같은 산업 혹은 섹터에 속한 경쟁사는?"
                              "(코스피, 뉴욕거래소 등 상장된 회사만 찾으세요. 반드시 회사명만 출력해주세요.)"})
    return peer_list

def find_peer_PERs_tool(company: str) -> None:
    """기업과 동종 업계의 Peer Group PER 평균을 찾습니다."""
    ticker = get_ticker(company)
    peer_list = find_peer(company)['answer']
    if ticker is None:
        return None
    
    peer_pers = {}
    for peer in peer_list:
        ticker = get_ticker(peer)
        if ticker is None:
            continue
        elif ".KS" in ticker:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            for i in range(4):
                earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm
            if trailingPERttm <0 :
                continue
            peer_pers[peer] = trailingPERttm
        else:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            try:
                for i in range(4):
                    earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            except:
                print(f"{peer}의 Net Income Common Stockholders 문제 발생")
                continue
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm*0.7 # 외국 주식의 경우 PER을 30% 할인
            if trailingPERttm <0 :
                continue
            if np.isnan(trailingPERttm):
                continue
            peer_pers[peer] = trailingPERttm

    valid_peer_pers = {}

    if state.PER < 0:
        valid_peer_pers = peer_pers
    else:
        for key, value in peer_pers.items():
            if value < 0.5 * state.PER or value > 2 * state.PER:
                continue
            else:
                valid_peer_pers[key] = value
                    
    print(valid_peer_pers)
    
    average_peer_per = sum(valid_peer_pers.values()) / len(valid_peer_pers)

    valid_peer_list = []
    for key, value in valid_peer_pers.items():
        valid_peer_list.append(key)

    state.peer_list = valid_peer_list
    state.average_peer_PER = average_peer_per

    return None

In [14]:
def find_PER_tool(company: str) -> None:
    """기업의 현재 PER(TTM)를 찾습니다."""
    ticker = get_ticker(company)
    if ticker is None:
        return None
    else:
        ticker = yf.Ticker(ticker)
        earning_ttm = 0
        for i in range(4):
            earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
        trailingPERttm = ticker.info["marketCap"]/earning_ttm
        state.PER = trailingPERttm
        return None


In [ ]:
find_PER_tool(state.company_name)
find_peer_PERs_tool(state.company_name)

In [ ]:
state.peer_list
state.average_peer_PER


In [ ]:
def segment_yoy_prediction_result(state:State) -> None:
    """사업부별 매출액 예측결과와 근거를 사업부별로 정리하여 작성"""
    segment_yoy_prediction_result = ""
    for segment in segments:
        segment_yoy_prediction_result += f"다음 분기 {segment} 사업부의 매출액은 yoy로 {state.segment[segment]['yoy_prediction']}% 변화할 것으로 예측됩니다.\n 직전 분기 {segment} 사업부의 매출액은 {state.segment[segment]['yoy']}% 변화했습니다.\n"
        segment_yoy_prediction_result += f"그 이유는 다음과 같습니다. {state.segment[segment]['yoy_prediction_reason']}\n"
        segment_yoy_prediction_result += f"이 예측은 다음과 같은 뉴스를 기반으로 이루어졌습니다. {state.segment[segment]['news_result']}\n"

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 각 사업부별 매출액 예측결과와 근거를 사업부별로 정리하여 작성하세요.
    근거를 서술할 때에는 근거가 된 뉴스를 반드시 언급하세요.
    다음 분기 매출액 yoy 증가율 예측결과와 직전분기 매출액 yoy 증가율을 비교하세요.

    사업부별 매출 예측 결과와 근거 : {segment_yoy_prediction_result}
    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm
    segment_yoy_prediction_result = chain.invoke({"segment_yoy_prediction_result": segment_yoy_prediction_result}).content
    state.result.update({"segment_yoy_prediction_result": segment_yoy_prediction_result})
    return None

segment_yoy_prediction_result(state)
print(state.result['segment_yoy_prediction_result'])

In [ ]:
def valuation_result(state:State) -> None:
    """목표 주가 컨센서스 작성"""
    ticker = get_ticker(state.company_name)
    ticker = yf.Ticker(ticker)
    estEarnings = income_stmt_cum.loc[income_stmt_cum['계정'] == '순이익','next_quarter'].values[0]
    shares = ticker.info.get("sharesOutstanding")
    estEPS = estEarnings/shares
    price_consensus = ticker.analyst_price_targets

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 {company_name}의 목표 주가를 설정하고 그 이유를 반드시 서술하세요.
    직전분기 매출 실적 리뷰 내용과 사업부별 매출 예측 결과와 근거는 반드시 서술하세요.
    목표 주가를 설정할 때에는 먼저, 현재 기업 PER과 경쟁사들의 평균 PER을 반드시 고려하여 기업의 목표 PER을 설정하세요.
    목표 PER과 추정 EPS를 곱하여 목표 주가를 설정하세요.

    직전 분기 매출 실적 리뷰: {current_quarter_review_result}

    사업부별 매출 예측 결과와 근거 : {segment_yoy_prediction_result}

    현재 기업 PER : {PER}

    기업 경쟁사 리스트 : {peer_list}

    현재 경쟁사들의 평균 PER : {average_peer_PER}

    추정 EPS : {EPS} (추정 매출액을 기반으로 계산된 결과입니다.)

    목표 주가를 제안한 이후, 목표 주가 컨센서스를 아래 내용을 참고하여 작성하세요.

    평균 목표 주가 : {avg_price_consensus}
    최소 목표 주가 : {low_price_consensus}
    최대 목표 주가 : {high_price_consensus}
    현재 주가 : {current_price}
    """


    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm
    valuation_result = chain.invoke({"company_name": state.company_name, 
                "current_quarter_review_result": state.result['sales_review'], 
                "segment_yoy_prediction_result": state.result['segment_yoy_prediction_result'], 
                "PER": state.PER, 
                "average_peer_PER": state.average_peer_PER,
                "peer_list": state.peer_list,
                "EPS": estEPS,
                "avg_price_consensus": price_consensus['mean'],
                "low_price_consensus": price_consensus['low'],
                "high_price_consensus": price_consensus['high'],
                "current_price": ticker.info.get("regularMarketPrice")})
    
    state.result.update({"valuation_result": valuation_result.content})
    return None

valuation_result(state)
print(state.result['valuation_result'])

In [ ]:
from datetime import datetime

title = f"{state.company_name} 리포트"
report = ""
report += f"{title}\n"
report += f"="*len(title)*2
report += f"\n\n"
for key, value in state.result.items():
    report += f"{value}\n\n"
    report += f"-"*50
    report += f"\n\n"

report += f"\n"
report += "본 자료는 사업보고서, 기사 등 신뢰할 수 있는 자료를 바탕으로 작성되었습니다.\n"
report += "그러나, AI의 의견이 반영되었으며 정확성이나 완전성을 보장할 수 없습니다.\n"
report += "본 자료는 참고용으로 투자자 자신의 판단과 책임하에 투자하시기 바랍니다.\n"
report += f"최종 리포트 작성일 : {datetime.now().strftime('%Y-%m-%d')}"

print(report)